# Predicting DWPC Query runtime ahead of time

In [8]:
import json

import matplotlib.pyplot
import pandas
import numpy
import seaborn
import mpld3

%matplotlib inline

In [9]:
path = 'data/all-features/metapaths.json'
with open(path) as fp:
    metapaths = json.load(fp)

In [10]:
auroc_df = pandas.read_table('data/all-features/auroc.tsv')
auroc_df.head(2)

,metapath,nonzero,seconds_per_query,auroc,auroc_permuted,delta_auroc,pval_auroc,length
0,CbGaD,0.312,0.0145,0.715,0.580,0.13500,0.000003,2
1,CbGdD,0.149,0.0136,0.512,0.515,-0.00332,0.921000,2


In [11]:
rows = [(item['abbreviation'], item['complexity_max'], item['complexity_mean']) for item in metapaths]
complexity_df = pandas.DataFrame(rows, columns=['metapath', 'complexity_max', 'complexity_mean'])
complexity_df = auroc_df.merge(complexity_df)
complexity_df['log10_seconds_per_query'] = numpy.log10(complexity_df['seconds_per_query'])

In [12]:
complexity_df.head(2)

,metapath,nonzero,seconds_per_query,auroc,auroc_permuted,delta_auroc,pval_auroc,length,complexity_max,complexity_mean,log10_seconds_per_query
0,CbGaD,0.312,0.0145,0.715,0.580,0.13500,0.000003,2,3.792672,0.620478,-1.838632
1,CbGdD,0.149,0.0136,0.512,0.515,-0.00332,0.921000,2,3.296665,1.206737,-1.866461


In [15]:
matplotlib.pyplot.figure(figsize=(10, 7))
ax = seaborn.regplot('complexity_mean', 'log10_seconds_per_query', data=complexity_df,
    lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'black'}, ci=False)
points = ax.collections[0]
labels = complexity_df.metapath.tolist()
tooltip = mpld3.plugins.PointLabelTooltip(points, labels)
mpld3.plugins.connect(ax.figure, tooltip)
mpld3.display()

In [16]:
matplotlib.pyplot.figure(figsize=(10, 7))
ax = seaborn.regplot('complexity_max', 'log10_seconds_per_query', data=complexity_df,
    lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'black'})
points = ax.collections[0]
labels = complexity_df.metapath.tolist()
tooltip = mpld3.plugins.PointLabelTooltip(points, labels)
mpld3.plugins.connect(ax.figure, tooltip)
mpld3.display()